# Representation Learning
The success of machine learning algorithms depends a lot on data representation. Representation learning has recently emerged as an alternative approach to feature extraction. In representation learning, features are extracted from unlabeled data by training a neural network on a secondary, supervised learning task. In this lab, we will apply representation learning techniques to textual and graph-structured data. The code that you will write make use of the Gensim and NetworkX libraries of Python.

## Part 1: Learning word representations
In the first part of the lab, we will use the [Word2Vec](http://www.cs.columbia.edu/~blei/seminar/2016_discrete_data/readings/MikolovSutskeverChenCorradoDean2013.pdf) model to learn high-quality word representations that capture several syntactic and semantic word relationship.

Throughout the entire session, we will be working with a subpart of the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. The OpinRank dataset contains reviews of cars and hotels. We will focus only on the hotel reviews. These reviews are stored in the `reviews_data.txt.gz` file. Each line in this file corresponds to a hotel review. Use the `load_data()` function defined below to read the data.

In [ ]:
import gzip

def load_data():
    data = list()
    with gzip.open ('data/reviews_data.txt.gz', 'rb') as f:
        for line in f:
            data.append(line.decode('utf-8', 'backslashreplace'))
    return data

#your code here

The `load_data()` function returns a list that contains all the hotel reviews. Print the second review of the dataset.

In [ ]:
#your code here

The documents that are contained in the dataset have already undergone some preprocessing. Therefore, we will only remove some punctuation marks, diacritics, and non letters, if any. Note that the input to the Word2Vec model has to be a list of lists (a list of sentences/documents where each sentence/document is a list of tokens). Therefore, we will also represent each document as a list of tokens. Use the `clean_str()` function defined below to preprocess each hotel review. Then, print the second preprocessed review.

In [ ]:
import re

def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().split()


#your code here

We will next train the CBOW model. Training of the CBOW model is very efficient since it does not involve any dense matrix multiplications. We will use Gensim's implementation of Word2Vec. We first need to initialize the Skipgram model (e.g., a [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec) object. The general syntax for initializing the model is the following:

```
model = Word2Vec(size=300, window=5, min_count=4, sg=1, workers=8)
```

where `size` corresponds to the dimensionality of the word embeddings, `window` is the maximum distance between the target word and its neighboring word within a sentence, `min_count` corresponds to the minimium frequency count of words, `sg` determines the type of the model (i.e., Skipgram or CBOW) and `workers` determines the number of CPU threads to use to train the model. Initialize a Word2Vec object and set the dimensionality of the word vectors equal to 100, the size of the window to 20, ignore all words with total frequency lower than 2, use the Skipgram model, and set the number of workers equal to the number of threads of your machine. Then, use the [build_vocab()](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.build_vocab) function of the Word2Vec object to build the vocabulary. This function takes as input a sequence of documents (e.g., list of lists) and extracts the vocabulary of the corpus.

In [ ]:
from gensim.models import Word2Vec

#your code here

After extracting the vocabulary, we can use the [train()](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.train) function of the Word2Vec object to start training the model. The general syntax of the function is the following:

```
model.train(input_data, total_examples=model.corpus_count, epochs=3) 
```

where `input_data` is the list of input documents, `total_examples` is the number of input documents, and `epochs` is the number of iterations over the input data. Use the `train()` function to train the model. Set the number of epochs to 5. Training time depends on the hardware of your machine. Note that it can take up to 10 minutes.

In [ ]:
#your code here

Next, we will experiment with the word embeddings learned by our model. First, we will get familiar with some interesting properties of word embeddings by performing some operations manually. The [most_similar()](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.most_similar) function returns the 10 terms that are most similar to a given word. The general syntax of the function is the following:

```
model.wv.most_similar(positive=input_word) 
```
 
Use the function to find the words that are similar to the word “France”.

In [ ]:
#your code here

You can specify the number of similar words to be retrieved by setting the `topn` parameter of the `most_similar()` function to some value. Use the function to retrieve the 20 words that are most similar to the word “football”.

In [ ]:
#your code here

Next, we will compute the similarity between terms. To calculate the similarity between two words, you can use the [similarity()](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.similarity) function which takes two words as input and computes their cosine similarity. The general syntax of the function is shown below:

```
model.wv.similarity(w1=input_word1, w2=input_word2)
```
 
Use the function to calculate the similarity between the words “dog” and “cat”.

In [ ]:
#your code here

Use the same function to compute the similarity between the words “rugby” and “man”, and the words “rugby” and “woman”. What do you observe?

In [ ]:
#your code here

From the above, it is clear that the word representations learned by neural networks are very interesting because the generated vectors explicitly encode many linguistic regularities and patterns. Many of these patterns can be represented as linear translations. You can find the terms that are most similar to such linear translations using the [most_similar()](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.most_similar) function. Use the function to obtain the 5 words that are most similar to the result of the following vector operation: “king” - “man” + “woman”. What do you observe?

In [ ]:
#your code here

Next, we will project the 200 most frequent words of our dataset into the two-dimensional space, and we will visualize them. The 200 most frequent words, correspond to the first 200 elements of the [index2entity](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.index2entity) list. Given a term, you can obtain its representation as follows:

```
vec = model.wv[word]
```

Store the representations of the 200 most frequent terms in a 200 $\times$ 100 matrix, and then use PCA/t-SNE to reduce their dimensionality. Then, visualize the terms in the two-dimensional space. 

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

%matplotlib inline

n = 200
vecs = 

#your code here

my_pca = PCA(n_components=10)
my_tsne = TSNE(n_components=2)

vecs_pca = my_pca.fit_transform(vecs)
vecs_tsne = my_tsne.fit_transform(vecs_pca)

fig, ax = plt.subplots()
ax.scatter(vecs_tsne[:,0], vecs_tsne[:,1],s=3)
for x, y, token in zip(vecs_tsne[:,0] , vecs_tsne[:,1], words):     
    ax.annotate(token, xy=(x, y), size=8)
fig.suptitle('t-SNE visualization of word embeddings',fontsize=30)
fig.set_size_inches(15,9)

## Part 2: Learning node representations
In recent years, representation learning on graphs has attracted increasing attention. The idea behind the representation learning approaches is to learn a mapping that embeds nodes as points in a low-dimensional vector space such that these representations reflect the structure of the graph. In this part of the lab, we will generate embeddings for the nodes of a graph using DeepWalk, a method that is inspired by recent advances in natural language processing.

Our dataset is a movie/actor network. This is a bipartite graph that consists of movies and of the actors that have played in them. More specifically, the nodes of the graph represent movies and actors, and there is an edge between a movie and an actor if the actor has played in that movie. The graph is stored in the `movies_actors_edgelist.txt` file as an edge list:

```
Toy Story (1995)	Tim Allen
Toy Story (1995)	Tom Hanks
Toy Story (1995)	Jim Varney
Toy Story (1995)	Don Rickles
Jumanji (1995)	Kirsten Dunst
Jumanji (1995)	Jonathan Hyde
Jumanji (1995)	Bradley Pierce
Jumanji (1995)	Robin Williams
Grumpier Old Men (1995)	Jack Lemmon
.
.
.
```

The network contains 14,612 nodes, 21,395 edges. Load the network data into an graph G, using the [read_edgelist()](https://networkx.github.io/documentation/stable/reference/readwrite/generated/networkx.readwrite.edgelist.read_edgelist.html) function of NetworkX. The general syntax of the function is the following:

```
G = nx.read_edgelist(path_to_file, comments='#', delimiter=None)
```

Note that, the delimeter used to separate values is the tab character '\t'.

In [ ]:
import networkx as nx

#your code here

Print the number of nodes and the number of the edges of the dataset (use the [`number_of_nodes()`](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.DiGraph.number_of_nodes.html#networkx.DiGraph.number_of_nodes) and [`number_of_edges()`](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.DiGraph.number_of_edges.html#networkx.DiGraph.number_of_edges) functions of NetworkX).

In [ ]:
#your code here

We will next implement [DeepWalk](http://www.perozzi.net/publications/14_kdd_deepwalk.pdf), an algorithm for generating node embeddings. DeepWalk builds on recent advances in unsupervised feature learning which have proven very successful in natural language processing. DeepWalk learns representations of a graph's vertices by running short random walks. These representations capture neighborhood similarity and community membership. The employed model is analogous to Skipgram:  given a vertex $v_i$, it estimates the likelihood of observing the previous and the following vertices visited in the random walk. Specifically, DeepWalk solves the following optimization problem:

$$
minimize_{\phi} \quad -\log \prod_{\substack{j=i-w \\ j \neq i}}^{i+w} P( v_j | \phi(v_i))
$$

where $\phi: V \rightarrow \mathbb{R}^d$ is a function that maps vertices to their embeddings, and $w$ is the size of the sliding window. Hence, given the embedding of a vertex, DeepWalk maximizes the probability of its neighbors in the walk.

Given a graph $G=(V,E)$ and a starting vertex $v_i$, a random walk of length $t$ is a stochastic process with random variables $w_{v_i}^{(1)}, w_{v_i}^{(2)},\ldots,w_{v_i}^{(t)}$ such that $w_{v_i}^{(1)} = v_i$ and $w_{v_i}^{(j)}$ is a a vertex chosen uniformly at random from the neighbors of vertex $w_{v_i}^{(j-1)}$. In other words, a random walk is a sequence of vertices: we first select a neighbor of $v_i$ at random, and move to this neighbor. Then, we select a neighbor of this new vertex at random, and move to it, etc.

Next, we will implement a function that given a graph and a starting vertex performs a random walk of specific length, and returns a list of the visited vertices. Fill in the body of the function `random_walk()`. Note that G is a NexworkX graph. You can use the [neighbors()](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.Graph.neighbors.html) function of NetworkX to get the neighbors of a vertex $v$.

In [ ]:
from random import randint

def random_walk(G, node, walk_length):
    # Starts from vertex "node" and performs a random walk of length "walk length". Returns a list of the visited vertices
    walk = [node]
    
    #your code here

    walk = [str(node) for node in walk]
    return walk

Next, you will implement a function that given a graph, it runs a number of random walks from each node of the graph, and returns a list of all random walks. Fill in the body of the function `generate_walks()`. Note that the function [nodes()](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.Graph.nodes.html) of NetworkX returns a list containing all the nodes of graph G.

In [ ]:
import numpy as np

def generate_walks(graph, num_walks, walk_length):
    # Runs "num_walks" random walks from each node, and returns a list of all random walk
    walks = list()
    
    #your code here

    return walks

After implementing the two functions, use the `generate_walks()` function to extract sequences of random walks from the graph. Start 10 walks from each node and set the length of the walks equal to 20.

In [ ]:
#your code here

Now, we can treat the generated walks as the equivalent of sentences and use the Gensim library to learn representations of the nodes of the movie/actor graph. To generate the embeddings, initialize a [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec) object and use its [build_vocab()](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.build_vocab) function to build the vocabulary. Set the dimensionality of the word vectors equal to 100, the size of the window to 5, use the Skipgram model, and set the number of workers equal to the number of threads of your machine. Then, use the [train()](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.train) function of the Word2Vec object to compute the node embeddings. Set the number of epochs to 5.

In [ ]:
#your code here

Use the [most_similar()](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.most_similar) function to obtain the 10 nodes that are most similar to the node “Keanu Reeves”. 

In [ ]:
#your code here

Use once again the [most_similar()](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.most_similar) function to obtain the 5 nodes that are most similar to the result of the following vector operation: “Eyes Wide Shut (1999)” - “Tom Cruise” + “Jack Nicholson”. What do you observe?

In [ ]:
#your code here